In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

IMG_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 30

def cargar_datos():
    df = pd.read_csv('final_full_df.csv')
    enfermedades = {
        'normal fundus': 0,
        'cataract': 1,
        'glaucoma': 2,
        'diabetic retinopathy': 3,
        'macular degeneration': 4,
        'hypertensive retinopathy': 5,
        'other': 6
    }
    df['label'] = df['A diagnostico'].map(enfermedades)
    df = df.dropna(subset=['label'])

    rutas = []
    labels = []
    for _, row in df.iterrows():
        img_path = os.path.join('Training Images', row['A ruta'])
        if os.path.exists(img_path):
            rutas.append(img_path)
            labels.append(row['label'])

    ros = RandomOverSampler()
    indices = np.arange(len(labels)).reshape(-1, 1)
    indices, _ = ros.fit_resample(indices, labels)

    X = []
    for i in indices.flatten():
        img = cv2.imread(rutas[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        X.append(img/255.0)

    X = np.array(X)
    y = to_categorical([labels[i] for i in indices.flatten()], num_classes=7)

    return X, y


def crear_modelo():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(7, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


def entrenar_y_guardar():
    X, y = cargar_datos()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = crear_modelo()

    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[
            callbacks.EarlyStopping(patience=5),
            callbacks.ModelCheckpoint('mejor_modelo.h5', save_best_only=True)
        ]
    )

    model.save('modelo_final.h5')
    print("Modelo guardado como 'modelo_final.h5'")

if __name__ == '__main__':
    entrenar_y_guardar()